In [ ]:
#In this cell, we are going to take an image, rotate it by 45 degrees, and then calculate its projection simply 
#by collapsing over the columns

# Importing libraries
from ipywidgets import *
from scipy import ndimage
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams.update(mpl.rcParamsDefault)
import numpy as np
%matplotlib notebook

# Import test image
# If this doesn't work make sure that you have an image at the path 'assets/ctimage.jpg'.
from matplotlib.image import imread
im = imread('assets/ctimage.jpg')[:,:,0].astype(np.float64)

f, ax = plt.subplots(1, 3, figsize=(8,8/3))

# Rotate image by 45 degrees counter clockwise
imR = ndimage.rotate(im, 45, reshape=False)

# Now we are going to take the 1D projection of this 2D rotated image by collapsing over the columns
proj = np.sum(imR, axis=0)

# Plot original
ax[0].imshow(im, cmap='gray')
# Diplay rotated image
ax[1].imshow(imR, cmap='gray')
# Plot the 1D projection
ax[2].plot(proj)

ax[0].set_title('Original')
ax[1].set_title('Rotated by 45 deg.')
ax[2].set_title('1D projection at angle 45')

plt.tight_layout()

In [ ]:
# In this cell, we are going to do the same thing interactively

f, ax = plt.subplots(1, 3, figsize=(8,8/3))
plt.tight_layout()

# Plot original
ax[0].imshow(im, cmap='gray')

# Make an updating function
def update(angle = .0):
    imR = ndimage.rotate(im, angle, reshape=False)
    ax[1].imshow(imR, cmap='gray')
    proj = np.sum(imR, axis=0)
    plt.cla()
    ax[2].plot(proj)
    
    
ax[0].set_title('Original')
ax[1].set_title('Rotated')
ax[2].set_title('1D projection')
  

# Add an interactive widget
interact(update, angle=(0.0,180.0,1.0));

In [ ]:
# In this cell we are going to generate and view a sinogram. We will use the same method as before
# except now we will save each projection into a sinogram array and instead of plotting line profiles
# of the projections we will visualize them collectively as a grayscale image

num_angles = 360

# Create array to hold the sinogram, which will have 360 views and 225 channels 
sino = np.zeros((num_angles,225))

f, ax = plt.subplots(1, 2, figsize=(8,8/3))

# Plot original
ax[0].imshow(im, cmap='gray')

for i in range(0,num_angles-1,1):
    angle = i
    imR = ndimage.rotate(im, angle, reshape=False)
    proj = np.sum(imR, axis=0)
    sino[i,:] = proj
    
ax[1].imshow(sino, cmap='gray')
ax[0].set_title('Image')
ax[1].set_title('Sinogram')

In [ ]:
# Here we will create a ramp filter by creating a vector of frequency values and taking its absolute value. 
# We will then use it to generate a filtered projection suitable for backprojection by multiplying it
# by the FT of the projection and taking an inverse FT.
# WARNING: In a more careful implementation we would (1) actually define the ramp filter as the inverse FT of 
# a particular sampled version of its spatial domain kernel and (2) zeropad projections to avoid DFT-related 
# periodicity artifacts. These implemnation details are spelled out in 
# A. C. Kak and Malcolm Slaney, Principles of Computerized Tomographic Imaging, IEEE Press, 1988.
# You can download the book (Chapter 3 discusses these details) at http://www.slaney.org/pct/pct-toc.html

v = np.fft.fftfreq(225) # Calculate frequency spacing 

ramp = np.abs(v)
ramlak = np.fft.ifftshift(np.fft.ifft(ramp))

f, ax = plt.subplots(1, 4, figsize=(8,3))


# Pick up the first projection from the sinogram and filter it
proj = sino[0,:]
projft = np.fft.fft(np.fft.fftshift(proj))
projfilt = np.real(np.fft.ifftshift(np.fft.ifft(projft*ramp)))

# Plot 
ax[0].plot(np.fft.fftshift(v), np.fft.fftshift(ramp), '-r', lw=0.5)
ax[1].plot(ramlak[100:124], lw=0.5)
ax[2].plot(proj, lw=0.5)
ax[3].plot(projfilt, lw=0.5)


ax[0].set_title('Ramp filter in v dom.')
ax[1].set_title('Ramp filter in x dom.')
ax[2].set_title('1D projection')
ax[3].set_title('1D filtered projection')

plt.tight_layout()

In [ ]:
#Here we will backproject a single projection (both unfiltered and filtered)

#Pick up one projection out of the sinogram
proj = sino[0,:]

#Now we will tile it into a 2D array with no variation in the vertical direction
bp = np.tile(proj,(225,1))
bpfilt = np.tile(projfilt,(225,1))

f, ax = plt.subplots(1, 3, figsize=(8,8/3))

# Plot original
ax[0].plot(proj)
ax[1].imshow(bp,cmap='gray')
ax[2].imshow(bpfilt,cmap='gray')

ax[0].set_title('Projection')
ax[1].set_title('BP of proj')
ax[2].set_title('BP of filtered proj')

plt.tight_layout()

In [ ]:
# Now we will reconstruct a whole image

# Define the array to hold the image
imrecon = np.zeros((225,225))

# Loop over projection angles
for i in range(0,num_angles-1,1):
    # Here we pick up each projection, filter it, and backproject it as above
    proj = sino[i,:]
    projft = np.fft.fft(np.fft.fftshift(proj))
    projfilt = np.real(np.fft.ifftshift(np.fft.ifft(projft*ramp)))
    bpfilt = np.tile(projfilt,(225,1))
    # Here we take the current backprojection image, rotate it and add it to the accumulating array
    imrecon = imrecon + ndimage.rotate(bpfilt, -i, reshape=False)
   

f, ax = plt.subplots(1, 4, figsize=(8,8/4))
plt.tight_layout()

# Plot original

# Set any negatives in the reconstructed image to zero
imrecon[imrecon < 0.0] = 0.0
ax[0].imshow(imrecon,cmap='gray')
ax[1].imshow(im,cmap='gray')

ax[0].set_title('Final recon')
ax[1].set_title('Starting image')

ax[2].plot(imrecon[112,:]/100.0)
ax[3].plot(im[112,:])